In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import time
import xarray as xr

from brainmodel_utils.core.constants import RIDGECV_ALPHA_CV
from brainmodel_utils.neural_mappers import PercentileNeuralMap
from brainmodel_utils.neural_mappers.utils import (
    generate_train_test_splits,
    convert_dict_to_tuple,
)

from brainmodel_utils.metrics.consistency import get_linregress_consistency


# Ensure PyTorch is using a GPU
if not torch.cuda.is_available():
    raise RuntimeError("PyTorch cannot find a GPU!")

device = torch.device("cuda:0")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
map_kwargs = {"map_type": "percentile"}
# turn it into immutable tuple to store as a key
map_kwargs_key = convert_dict_to_tuple(map_kwargs)

dim_trials = 60
dim_times = 20
dim_units_source = 5*13000
dim_units_target = 5*1000

window_size = 10


source_resp = np.random.normal(0,1,size = (dim_trials,dim_times,dim_units_source))
target_resp = np.random.normal(0,1,size = (dim_trials,dim_times,dim_units_target))

coords_source = {
    "trials": np.arange(dim_trials),  # replace with your actual coordinates
    "time": np.linspace(-window_size,window_size,dim_times),    # replace with your actual coordinates
    "units": np.arange(dim_units_source)   # replace with your actual coordinates
}

coords_target = {
    "trials": np.arange(dim_trials),  # replace with your actual coordinates
    "time": np.linspace(-window_size,window_size,dim_times),    # replace with your actual coordinates
    "units": np.arange(dim_units_target)   # replace with your actual coordinates
}

source_resp = xr.DataArray(source_resp, dims=("trials", "time", "units"), coords = coords_source)
target_resp = xr.DataArray(target_resp, dims=("trials", "time", "units"), coords = coords_target)


results = get_linregress_consistency(
    source=source_resp,
    target=target_resp,
    map_kwargs=map_kwargs,
    num_bootstrap_iters=1,
    num_parallel_jobs=1,
    splits=None,
    train_frac=0.5,
    num_train_test_splits=5,
)



/om2/user/leokoz8/envs/zfa/lib/python3.10/site-packages/brainmodel_utils/metrics/consistency.py:41: RuntimeWarning: invalid value encountered in double_scalars
  denom_sb = (r_xx_sb * r_yy_sb) ** 0.5
/om2/user/leokoz8/envs/zfa/lib/python3.10/site-packages/brainmodel_utils/metrics/consistency.py:41: RuntimeWarning: invalid value encountered in double_scalars
  denom_sb = (r_xx_sb * r_yy_sb) ** 0.5
/om2/user/leokoz8/envs/zfa/lib/python3.10/site-packages/brainmodel_utils/metrics/consistency.py:41: RuntimeWarning: invalid value encountered in double_scalars
  denom_sb = (r_xx_sb * r_yy_sb) ** 0.5
/om2/user/leokoz8/envs/zfa/lib/python3.10/site-packages/brainmodel_utils/metrics/consistency.py:41: RuntimeWarning: invalid value encountered in double_scalars
  denom_sb = (r_xx_sb * r_yy_sb) ** 0.5
/om2/user/leokoz8/envs/zfa/lib/python3.10/site-packages/brainmodel_utils/metrics/consistency.py:41: RuntimeWarning: invalid value encountered in double_scalars
  denom_sb = (r_xx_sb * r_yy_sb) ** 0.5


In [13]:
np.nanmedian(results['test']['r_xy_n_sb'])

0.005006196349394159

In [14]:
results

{'train': defaultdict(list,
             {'r_xy_n_sb': <xarray.DataArray (trial_bootstrap_iters: 1, train_test_splits: 5, units: 5000)>
              array([[[        nan,  3.35303549,  1.90405728, ..., 10.73645743,
                               nan,  3.30793901],
                      [ 4.10560169,  1.42522502,  2.32464671, ...,  1.55816244,
                        4.98614438,  6.81069866],
                      [ 3.97898317,  5.41233701,  1.27417321, ...,  3.27050267,
                        5.61687754,  1.55249193],
                      [ 0.89120992,  1.71751782,  2.196423  , ...,  1.26077234,
                        1.78840098,  1.26646383],
                      [ 2.49897532,  2.85746088,  1.97568205, ...,  1.76597916,
                        6.28631894,  1.60492225]]])
              Coordinates:
                * units    (units) int64 0 1 2 3 4 5 6 ... 4993 4994 4995 4996 4997 4998 4999
              Dimensions without coordinates: trial_bootstrap_iters, train_test_splits,
   

In [11]:
# Random data generation
size = 1000

a_np = np.random.randn(size, size)
b_np = np.random.randn(size, size)

# Convert numpy arrays to PyTorch tensors on GPU
a_torch = torch.tensor(a_np, device=device).float()
b_torch = torch.tensor(b_np, device=device).float()

# Warm-up to move data to GPU and compile any GPU operations
_ = np.dot(a_np, b_np)
_ = torch.mm(a_torch, b_torch)

# Timing numpy dot product
start_time = time.time()
np_result = np.dot(a_np, b_np)
np_time = time.time() - start_time

# Timing PyTorch dot product on GPU
start_time = time.time()
torch_result = torch.mm(a_torch, b_torch)
torch_time = time.time() - start_time

print(f"Numpy dot product time (on CPU): {np_time:.6f} seconds")
print(f"PyTorch dot product time (on GPU): {torch_time:.6f} seconds")

# Optional: compare results to ensure they're approximately equal
# Move the torch result back to CPU and convert to numpy for comparison
torch_result_np = torch_result.cpu().numpy()
assert np.allclose(np_result, torch_result_np, atol=1e-4), "The results are not close enough!"


62.5

In [6]:
0.3*5

1.5